# Setup

In [1]:
!pip install geopandas matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 67.5 MB/s eta 0:00:00


In [49]:
# Loading/Importing the libraries
from branca import colormap
from folium import IFrame
from folium import plugins
from folium.plugins import FloatImage
from google.colab import drive
from IPython.display import display
import branca.colormap as cm
import folium
import geopandas as gpd
import imageio
import ipywidgets as widgets
import plotly.io as pio
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import pandas as pd
import sqlite3

In [3]:
# mounting the drive
drive.mount("/content/drive", force_remount=True)
file_base = "https://docs.google.com/uc?id="

Mounted at /content/drive


In [4]:
# connect to database
con = sqlite3.connect("project.db")

In [5]:
# constants
YEAR_START = 1993
YEAR_END = 2014

# Data Munging & Database Population

## Vehicle Inventory

In [6]:
# read data: County Wise Vehicle Inventory
file_vehicles = file_base + "1vRvLqovdvIyDfZPuFxomWI1eUOKm16NX"
df_vehicles = pd.read_excel(file_vehicles)
df_vehicles.head()

,state_abbr,geoid,county_id,county_name,fuel_type_org,fuel_type,class,before 1980,1980-99,1990-99,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,AL,1001,1001,Autauga,BI,Other/Unknown,Car,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AL,1001,1001,Autauga,BI,Other/Unknown,Truck,NaN,NaN,0.000019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AL,1001,1001,Autauga,DIES,Diesel vehicle,Car,NaN,0.000190,0.000038,...,0.000114,0.000152,0.000229,0.000209,0.000267,0.000362,0.000171,NaN,NaN,NaN
3,AL,1001,1001,Autauga,DIES,Diesel vehicle,Truck,NaN,0.001143,0.005447,...,0.000457,0.000381,0.001086,0.000971,0.000686,0.000648,0.001733,0.001638,0.000038,NaN
4,AL,1001,1001,Autauga,DIES,Diesel vehicle,Unknown,0.000209,0.000019,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# - Drops the columns 'county_name', 'county_id', 'geoid', 'class', 'fuel_type_org', 'before 1980', and '1980-99' from the DataFrame df_vehicles using the drop() method. The axis=1 parameter indicates that the columns are to be dropped.
# - Combines the fuel type values in the 'fuel_type' column using the replace() method. Specific replacements are defined using a dictionary where the original values are the keys and the desired replacement values are the corresponding values.
# - Groups the DataFrame df_vehicles_2 by 'state_abbr' and 'fuel_type' using the groupby() method.
# - Calculates the mean for each column within each group using the mean() method.
# - Resets the index of the resulting grouped DataFrame using the reset_index() method.
# - Prints the aggregated DataFrame df_vehicles_aggregated.

# The resulting DataFrame df_vehicles_aggregated contains the mean values for each column, grouped by 'state_abbr' and 'fuel_type'.


# Drop the 'county_name', 'county_id', 'geoid', 'class', 'fuel_type_org', 'before 1980', '1980-99' columns
df_vehicles_2 = df_vehicles.drop(['county_name', 'county_id', 'geoid', 'class', 'fuel_type_org', 'before 1980', '1980-99'], axis=1)

# Combine fuel type values
df_vehicles_2['fuel_type'] = df_vehicles_2['fuel_type'].replace({
    'Diesel vehicle': 'Gas Powered',
    'Electric vehicle': 'Clean Fuel',
    'Flex fuel vehicle': 'Gas Powered',
    'Gasoline vehicle': 'Gas Powered',
    'Hybrid electric vehicle': 'Clean Fuel',
    'Hydrogen fuel cell vehicle': 'Clean Fuel',
    'Other/Unknown': 'Other/Unknown',
    'Plug-in hybrid electric vehicle': 'Clean Fuel'
})

# Group by state and calculate the mean for each column
df_vehicles_aggregated = df_vehicles_2.groupby(['state_abbr', 'fuel_type']).mean().reset_index()

# Print the aggregated DataFrame
df_vehicles_aggregated

,state_abbr,fuel_type,1990-99,2000,2001,2002,2003,2004,2005,2006,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,AK,Clean Fuel,0.000004,0.000016,0.000052,0.000060,0.000084,0.000148,0.000306,0.000385,...,0.000249,0.000344,0.000120,0.000193,0.000202,0.000253,0.000185,0.000109,0.000028,NaN
1,AK,Gas Powered,0.039917,0.007150,0.007721,0.007150,0.007188,0.007593,0.007048,0.007914,...,0.003944,0.005134,0.005398,0.005084,0.005094,0.005419,0.005273,0.004353,0.000475,NaN
2,AK,Other/Unknown,0.000858,0.000137,0.000192,0.000121,0.000218,0.000233,0.000262,0.000290,...,0.000119,0.000112,0.000068,0.000100,0.000121,0.000127,0.000113,0.000119,0.000231,0.0
3,AL,Clean Fuel,0.000007,0.000008,0.000014,0.000014,0.000034,0.000049,0.000133,0.000200,...,0.000103,0.000233,0.000154,0.000218,0.000218,0.000196,0.000180,0.000122,0.000034,NaN
4,AL,Gas Powered,0.025934,0.005595,0.005704,0.006399,0.006623,0.007064,0.007124,0.007912,...,0.004291,0.005105,0.006076,0.006312,0.006761,0.006570,0.006864,0.006033,0.001112,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,WV,Gas Powered,0.023716,0.005726,0.005692,0.006479,0.006823,0.007294,0.007190,0.007694,...,0.004976,0.005918,0.007528,0.007872,0.008347,0.008559,0.008872,0.007227,0.001022,NaN
149,WV,Other/Unknown,0.001331,0.000043,0.000257,0.000374,0.000028,0.000033,0.000042,0.000036,...,0.000030,0.000020,0.000037,0.000032,0.000043,0.000055,0.000058,0.000067,0.000043,0.0
150,WY,Clean Fuel,0.000006,0.000018,0.000020,0.000026,0.000043,0.000107,0.000274,0.000350,...,0.000207,0.000378,0.000186,0.000198,0.000214,0.000221,0.000197,0.000153,0.000019,NaN
151,WY,Gas Powered,0.026850,0.005153,0.005957,0.006112,0.005866,0.006675,0.006788,0.008081,...,0.004652,0.005150,0.006939,0.007318,0.006779,0.007371,0.007822,0.004505,0.000468,NaN


In [8]:
# - Checks if the year column exists in the df_vehicles_aggregated DataFrame. If the column does not exist, a new column is created with the year as the column name.
# - Assigns the value from the "1990-99" column to the corresponding new year column for the current row.

# After iterating over all rows and years, the code constructs a new DataFrame df_vehicles by selecting specific columns from df_vehicles_aggregated based on the defined column_headings.

for index, row in df_vehicles_aggregated.iterrows():
    for year in range(YEAR_START, 2000):
        # Create new year columns if they don't exist
        if year not in df_vehicles_aggregated.columns:
            df_vehicles_aggregated[year] = None

        # Assign the value from "1990-99" column to the corresponding new year column
        df_vehicles_aggregated.at[index, year] = row["1990-99"]

column_headings = df_vehicles_aggregated.columns[:2].tolist() + list(range(YEAR_START, 2000)) + df_vehicles_aggregated.columns[3:-11].tolist()
df_vehicles = df_vehicles_aggregated[column_headings]
df_vehicles

,state_abbr,fuel_type,1993,1994,1995,1996,1997,1998,1999,2000,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,AK,Clean Fuel,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000016,...,0.000306,0.000385,0.000580,0.000515,0.000249,0.000344,0.000120,0.000193,0.000202,0.000253
1,AK,Gas Powered,0.039917,0.039917,0.039917,0.039917,0.039917,0.039917,0.039917,0.007150,...,0.007048,0.007914,0.007091,0.007628,0.003944,0.005134,0.005398,0.005084,0.005094,0.005419
2,AK,Other/Unknown,0.000858,0.000858,0.000858,0.000858,0.000858,0.000858,0.000858,0.000137,...,0.000262,0.000290,0.000485,0.000257,0.000119,0.000112,0.000068,0.000100,0.000121,0.000127
3,AL,Clean Fuel,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000008,...,0.000133,0.000200,0.000242,0.000156,0.000103,0.000233,0.000154,0.000218,0.000218,0.000196
4,AL,Gas Powered,0.025934,0.025934,0.025934,0.025934,0.025934,0.025934,0.025934,0.005595,...,0.007124,0.007912,0.008398,0.007094,0.004291,0.005105,0.006076,0.006312,0.006761,0.006570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,WV,Gas Powered,0.023716,0.023716,0.023716,0.023716,0.023716,0.023716,0.023716,0.005726,...,0.007190,0.007694,0.008085,0.007838,0.004976,0.005918,0.007528,0.007872,0.008347,0.008559
149,WV,Other/Unknown,0.001331,0.001331,0.001331,0.001331,0.001331,0.001331,0.001331,0.000043,...,0.000042,0.000036,0.000049,0.000040,0.000030,0.000020,0.000037,0.000032,0.000043,0.000055
150,WY,Clean Fuel,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000018,...,0.000274,0.000350,0.000455,0.000292,0.000207,0.000378,0.000186,0.000198,0.000214,0.000221
151,WY,Gas Powered,0.02685,0.02685,0.02685,0.02685,0.02685,0.02685,0.02685,0.005153,...,0.006788,0.008081,0.008123,0.007560,0.004652,0.005150,0.006939,0.007318,0.006779,0.007371


- These stats for the State wise Vehicle Inventory only account for the sale of light motor vehicles in the state during a particular year, and thus, they may not add up to 100%.
- We want to restrict our dataset to just light motor vehicles.
- Thus, we calculate the relative percentages and extrapolate the values to 100%.

In [9]:
# get_current_total_percentage(state, year): This function takes a state abbreviation (state) and a year (year) as input.
# It filters the df_vehicles DataFrame for the specified state and calculates the total percentage value for the given year.
# The total percentage is the sum of all fuel type percentages for the specified state and year.

def get_current_total_percentage(state, year):
    return df_vehicles[df_vehicles['state_abbr'] == state][year].sum()


# get_current_percentage(state, year, fuel_type): This function takes a state abbreviation (state), a year (year), and a fuel type (fuel_type) as input.
# It filters the df_vehicles DataFrame for the specified state, fuel type, and year, and retrieves the percentage value for that specific fuel type in the given year and state.
def get_current_percentage(state, year, fuel_type):
    return list(df_vehicles[(df_vehicles['state_abbr'] == state) & (df_vehicles['fuel_type'] == fuel_type)][year])[0]


# This function takes a state abbreviation (state), a year (year), and a fuel type (fuel_type) as input.
# It calculates the extrapolated percentage for the specified fuel type in the given year and state.
# The extrapolated percentage is calculated by dividing the current percentage for the fuel type in the specified year by the total percentage for that year and state, and then multiplying it by 100.
# The result is rounded to three decimal places.
def get_extrapolated_percentage(state, year, fuel_type):
    total_percentage = get_current_total_percentage(state, year)
    current_percentage = get_current_percentage(state, year, fuel_type)
    return round(((current_percentage / total_percentage) * 100), 3)

In [10]:
# The code provided creates a list called data and iterates over each row in the df_vehicles DataFrame.
# For each row, it retrieves the state abbreviation and fuel type values.
# Then, a new row is created with the state abbreviation and fuel type as the first two elements.

# Next, the code iterates over the range of years from YEAR_START to YEAR_END (inclusive), and for each year, it calls the get_extrapolated_percentage function to retrieve the extrapolated percentage for the given state, year, and fuel type.
# The extrapolated percentage is then appended to the new row.

# Finally, the new row is appended to the data list.

# The resulting data list will contain rows with the following structure:
# [state_abbr, fuel_type, extrapolated_percentage_1, extrapolated_percentage_2, ..., extrapolated_percentage_n]

# Each row represents a combination of state, fuel type, and the extrapolated percentages for each year from YEAR_START to YEAR_END.

data = []

for _, row in df_vehicles.iterrows():
    state, fuel_type = row['state_abbr'], row['fuel_type']
    new_row = [state, fuel_type]
    for year in range(YEAR_START, YEAR_END + 1):
        new_row.append(get_extrapolated_percentage(state, year, fuel_type))
    data.append(new_row)

In [11]:
# The code provided creates a new DataFrame called df_vehicles using the data list and the column_headings list as column names.
# The resulting DataFrame will have the extrapolated percentage values for each state, fuel type, and year.

df_vehicles = pd.DataFrame(data, columns=column_headings)
df_vehicles

,state_abbr,fuel_type,1993,1994,1995,1996,1997,1998,1999,2000,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,AK,Clean Fuel,0.009,0.009,0.009,0.009,0.009,0.009,0.009,0.217,...,4.012,4.478,7.116,6.131,5.776,6.159,2.153,3.585,3.728,4.360
1,AK,Gas Powered,97.886,97.886,97.886,97.886,97.886,97.886,97.886,97.903,...,92.551,92.145,86.937,90.805,91.459,91.844,96.636,94.561,94.030,93.443
2,AK,Other/Unknown,2.105,2.105,2.105,2.105,2.105,2.105,2.105,1.880,...,3.437,3.377,5.947,3.063,2.765,1.997,1.211,1.854,2.242,2.197
3,AL,Clean Fuel,0.026,0.026,0.026,0.026,0.026,0.026,0.026,0.139,...,1.810,2.440,2.776,2.136,2.313,4.305,2.462,3.316,3.105,2.848
4,AL,Gas Powered,98.327,98.327,98.327,98.327,98.327,98.327,98.327,98.139,...,97.112,96.567,96.477,97.090,96.824,94.379,96.999,95.889,96.272,95.624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,WV,Gas Powered,94.688,94.688,94.688,94.688,94.688,94.688,94.688,98.971,...,97.305,97.209,96.099,96.663,97.374,94.616,96.982,96.483,96.636,96.601
149,WV,Other/Unknown,5.312,5.312,5.312,5.312,5.312,5.312,5.312,0.748,...,0.574,0.458,0.585,0.497,0.583,0.323,0.474,0.393,0.502,0.618
150,WY,Clean Fuel,0.022,0.022,0.022,0.022,0.022,0.022,0.022,0.336,...,3.818,4.100,5.201,3.635,4.153,6.775,2.571,2.605,3.032,2.881
151,WY,Gas Powered,98.248,98.248,98.248,98.248,98.248,98.248,98.248,98.532,...,94.501,94.529,92.936,94.196,93.429,92.279,96.027,96.406,95.843,96.254


In [12]:
# df_vehicles DataFrame is written to a table named "vehicles" in the SQLite database represented by the con connection object.
# The if_exists argument is set to "replace", which means that if the "vehicles" table already exists, it will be replaced with the new data.
# The index argument is set to False, indicating that the DataFrame index should not be written as a column in the table.

df_vehicles.to_sql("vehicles", con, if_exists = "replace", index = False)

153

In [13]:
# Verify the vehicles table in the database has been populated correctly.

# Takes an SQL query as input and executes it against the specified database connection object (con in this case).
# It returns a DataFrame containing the results of the query.

query_vehicle_head = """
SELECT *
FROM vehicles
LIMIT 6;
"""

df_query_vehicle_head = pd.read_sql_query(query_vehicle_head, con)
df_query_vehicle_head

,state_abbr,fuel_type,1993,1994,1995,1996,1997,1998,1999,2000,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,AK,Clean Fuel,0.009,0.009,0.009,0.009,0.009,0.009,0.009,0.217,...,4.012,4.478,7.116,6.131,5.776,6.159,2.153,3.585,3.728,4.360
1,AK,Gas Powered,97.886,97.886,97.886,97.886,97.886,97.886,97.886,97.903,...,92.551,92.145,86.937,90.805,91.459,91.844,96.636,94.561,94.030,93.443
2,AK,Other/Unknown,2.105,2.105,2.105,2.105,2.105,2.105,2.105,1.880,...,3.437,3.377,5.947,3.063,2.765,1.997,1.211,1.854,2.242,2.197
3,AL,Clean Fuel,0.026,0.026,0.026,0.026,0.026,0.026,0.026,0.139,...,1.810,2.440,2.776,2.136,2.313,4.305,2.462,3.316,3.105,2.848
4,AL,Gas Powered,98.327,98.327,98.327,98.327,98.327,98.327,98.327,98.139,...,97.112,96.567,96.477,97.090,96.824,94.379,96.999,95.889,96.272,95.624
5,AL,Other/Unknown,1.647,1.647,1.647,1.647,1.647,1.647,1.647,1.722,...,1.078,0.994,0.747,0.774,0.863,1.316,0.539,0.795,0.623,1.528


## Gasoline Prices

In [14]:
# # read data: Gasoline prices per State
# # values: Gasoline prices per State over the years
file_gas = file_base + "1hvAtPywtbh-jVLilwenykzxGca_o6Wq1"
df_gas = pd.read_excel(file_gas, header=2)
df_gas.head()

,State,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,3.18,3.25,3.21,3.46,4.76,5.15,5.36,5.69,6.09,...,34.44,36.19,34.81,33.68,27.66,24.07,26.82,30.73,31.19,26.53
1,AL,2.82,2.85,2.95,3.08,4.14,4.26,4.70,5.12,5.27,...,27.53,27.99,27.16,25.89,18.17,16.31,18.50,20.40,18.97,15.51
2,AR,2.74,2.84,2.77,2.96,4.09,4.60,4.82,5.15,5.30,...,27.86,28.32,27.48,26.20,18.38,16.50,18.72,20.64,19.19,15.69
3,AZ,2.80,2.87,2.83,3.06,4.12,4.62,4.84,5.09,5.42,...,27.70,29.11,28.00,27.09,22.25,19.36,21.58,24.72,25.09,21.34
4,CA,2.80,2.84,2.79,3.08,4.47,4.84,5.04,5.33,5.48,...,30.51,32.24,31.08,29.99,25.47,22.01,24.37,28.09,29.10,24.79


In [15]:
# # read data: Gasoline prices per State per Capita Expenditure
# # values: Gasoline prices per State over the years
file_gas_per_capita = file_base + "1uu-ojSLZGqHaJ3KuG02O6QkmadKtk_xx"
df_gas_per_capita = pd.read_excel(file_gas_per_capita)
df_gas_per_capita.head()

,State,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,144.2,153.7,191.8,176.0,259.9,300.4,330.1,358.8,358.5,...,1603.4,1669.6,1547.8,1563.4,1302.8,1141.4,1239.9,1411.4,1414.3,1069.2
1,AL,158.7,167.2,181.0,197.7,264.6,274.7,314.0,350.1,366.4,...,1782.9,1784.1,1741.1,1654.9,1208.8,1113.9,1242.0,1359.8,1301.1,1051.8
2,AR,167.6,179.9,185.6,203.4,276.1,308.6,339.8,365.0,380.1,...,1616.6,1637.7,1559.2,1527.2,1088.2,1008.9,1136.1,1228.3,1165.3,887.1
3,AZ,176.9,182.5,189.0,210.6,260.1,294.1,313.3,339.3,366.9,...,1344.8,1382.5,1343.6,1289.5,1097.6,972.1,1073.3,1234.1,1239.9,955.4
4,CA,157.4,160.9,165.5,186.9,261.1,285.0,304.7,333.4,350.6,...,1418.7,1471.1,1424.4,1366.2,1185.5,1036.6,1148.4,1316.3,1343.0,919.2


In [16]:
print(df_gas.columns)

Index(['State',    1970,    1971,    1972,    1973,    1974,    1975,    1976,
          1977,    1978,    1979,    1980,    1981,    1982,    1983,    1984,
          1985,    1986,    1987,    1988,    1989,    1990,    1991,    1992,
          1993,    1994,    1995,    1996,    1997,    1998,    1999,    2000,
          2001,    2002,    2003,    2004,    2005,    2006,    2007,    2008,
          2009,    2010,    2011,    2012,    2013,    2014,    2015,    2016,
          2017,    2018,    2019,    2020],
      dtype='object')


In [17]:
print(df_gas_per_capita.columns)

Index(['State',    1970,    1971,    1972,    1973,    1974,    1975,    1976,
          1977,    1978,    1979,    1980,    1981,    1982,    1983,    1984,
          1985,    1986,    1987,    1988,    1989,    1990,    1991,    1992,
          1993,    1994,    1995,    1996,    1997,    1998,    1999,    2000,
          2001,    2002,    2003,    2004,    2005,    2006,    2007,    2008,
          2009,    2010,    2011,    2012,    2013,    2014,    2015,    2016,
          2017,    2018,    2019,    2020],
      dtype='object')


In [18]:
# clean data: Gasoline
# drop unnecessary columns, null values, and irrelevant years
# Initialize list for columns to drop
gasColumnsToDrop = []

# Add year columns outside the range of YEAR_START to YEAR_END
gasColumnsToDrop.extend([col for col in df_gas.columns
                         if isinstance(col, int) and (col < YEAR_START or col > YEAR_END)])

# Drop the columns and null values
df_gas.drop(columns=gasColumnsToDrop, inplace=True)
df_gas.dropna(inplace=True)

In [19]:
# clean data: Gasoline Per Capita Expenditure
# drop unnecessary columns, null values, and irrelevant years
# Initialize list for columns to drop
gasColumnsToDrop = []

# Add year columns outside the range of YEAR_START to YEAR_END
gasColumnsToDrop.extend([col for col in df_gas_per_capita.columns
                         if isinstance(col, int) and (col < YEAR_START or col > YEAR_END)])

# Drop the columns and null values
df_gas_per_capita .drop(columns=gasColumnsToDrop, inplace=True)
df_gas_per_capita .dropna(inplace=True)

In [20]:
df_gas.head()

,State,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,AK,10.05,10.54,10.90,11.74,12.02,10.20,10.08,12.87,13.30,...,19.05,21.54,22.89,29.84,23.30,27.98,34.44,36.19,34.81,33.68
1,AL,8.68,8.62,8.93,9.36,9.42,8.18,8.77,11.42,10.76,...,17.59,19.76,21.55,25.80,18.07,21.72,27.53,27.99,27.16,25.89
2,AR,8.62,8.64,8.77,9.43,9.33,8.00,8.52,11.38,10.93,...,17.49,19.73,22.12,25.56,17.84,21.97,27.86,28.32,27.48,26.20
3,AZ,9.64,9.61,9.66,10.57,10.60,8.90,9.68,12.20,11.64,...,18.74,20.70,22.22,25.89,18.74,22.50,27.70,29.11,28.00,27.09
4,CA,9.14,9.13,9.27,10.03,10.27,9.01,10.52,12.55,12.27,...,18.96,21.47,23.34,26.96,20.56,24.76,30.51,32.24,31.08,29.99


In [21]:
df_gas_per_capita.head()

,State,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,AK,522.9,595.7,671.0,677.2,644.1,576.9,539.1,636.8,696.8,...,1016.3,1122.7,1198.7,1486.6,1138.2,1365.3,1603.4,1669.6,1547.8,1563.4
1,AL,558.1,561.5,600.2,619.2,625.1,554.6,593.8,762.5,722.7,...,1256.7,1405.0,1524.6,1745.2,1210.5,1454.7,1782.9,1784.1,1741.1,1654.9
2,AR,557.9,557.9,578.4,612.6,619.6,527.1,563.3,735.6,701.9,...,1126.4,1253.1,1395.9,1550.5,1099.0,1330.4,1616.6,1637.7,1559.2,1527.2
3,AZ,532.4,533.4,534.9,593.2,569.6,499.6,549.9,694.0,671.9,...,1124.7,1233.6,1296.8,1384.4,953.6,1123.4,1344.8,1382.5,1343.6,1289.5
4,CA,470.9,465.4,477.2,519.7,531.5,468.8,551.8,658.5,651.3,...,1047.7,1184.0,1260.4,1370.4,1010.1,1194.2,1418.7,1471.1,1424.4,1366.2


In [22]:
# save to database: Gasoline Prices
df_gas.to_sql("gasoline", con, if_exists = "replace", index = False)

52

In [23]:
# save to database: Gasoline Prices
df_gas_per_capita.to_sql("gasoline_per_capita", con, if_exists = "replace", index = False)

52

In [24]:
# verify database population
query_gasoline_head = """
SELECT *
FROM gasoline
LIMIT 6;
"""

df_query_gasoline_head = pd.read_sql_query(query_gasoline_head, con)
df_query_gasoline_head

,State,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,AK,10.05,10.54,10.90,11.74,12.02,10.20,10.08,12.87,13.30,...,19.05,21.54,22.89,29.84,23.30,27.98,34.44,36.19,34.81,33.68
1,AL,8.68,8.62,8.93,9.36,9.42,8.18,8.77,11.42,10.76,...,17.59,19.76,21.55,25.80,18.07,21.72,27.53,27.99,27.16,25.89
2,AR,8.62,8.64,8.77,9.43,9.33,8.00,8.52,11.38,10.93,...,17.49,19.73,22.12,25.56,17.84,21.97,27.86,28.32,27.48,26.20
3,AZ,9.64,9.61,9.66,10.57,10.60,8.90,9.68,12.20,11.64,...,18.74,20.70,22.22,25.89,18.74,22.50,27.70,29.11,28.00,27.09
4,CA,9.14,9.13,9.27,10.03,10.27,9.01,10.52,12.55,12.27,...,18.96,21.47,23.34,26.96,20.56,24.76,30.51,32.24,31.08,29.99
5,CO,9.67,9.91,9.80,10.48,10.55,8.94,9.74,12.42,12.43,...,18.32,20.72,22.98,26.09,18.61,21.99,27.95,28.65,28.17,27.52


In [25]:
# verify database population
query_gasoline_per_capita_head = """
SELECT *
FROM gasoline_per_capita
LIMIT 6;
"""

df_query_gasoline_per_capita_head = pd.read_sql_query(query_gasoline_per_capita_head, con)
df_query_gasoline_per_capita_head

,State,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,AK,522.9,595.7,671.0,677.2,644.1,576.9,539.1,636.8,696.8,...,1016.3,1122.7,1198.7,1486.6,1138.2,1365.3,1603.4,1669.6,1547.8,1563.4
1,AL,558.1,561.5,600.2,619.2,625.1,554.6,593.8,762.5,722.7,...,1256.7,1405.0,1524.6,1745.2,1210.5,1454.7,1782.9,1784.1,1741.1,1654.9
2,AR,557.9,557.9,578.4,612.6,619.6,527.1,563.3,735.6,701.9,...,1126.4,1253.1,1395.9,1550.5,1099.0,1330.4,1616.6,1637.7,1559.2,1527.2
3,AZ,532.4,533.4,534.9,593.2,569.6,499.6,549.9,694.0,671.9,...,1124.7,1233.6,1296.8,1384.4,953.6,1123.4,1344.8,1382.5,1343.6,1289.5
4,CA,470.9,465.4,477.2,519.7,531.5,468.8,551.8,658.5,651.3,...,1047.7,1184.0,1260.4,1370.4,1010.1,1194.2,1418.7,1471.1,1424.4,1366.2
5,CO,529.5,546.3,551.1,599.3,597.7,506.9,564.2,708.0,724.9,...,1053.9,1176.7,1285.2,1371.3,960.3,1128.7,1392.6,1406.7,1393.6,1364.9


## CO2 Emissions

In [26]:
# read data: CO2 emissions
# values: million metric tons of energy-related CO2
file_co2 = file_base + "1N-5WW6KgTFslOjCyiwOlAb9AdBrf6Grb"
df_co2 = pd.read_excel(file_co2)

# values: metric tons of energy-related CO2 per person
file_co2_per_capita = file_base + "1MTqZYZ1K6s7CWZ8Mqwlfdu8OY52ZkwOQ"
df_co2_per_capita = pd.read_excel(file_co2_per_capita)

df_co2.head()

,State,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2015,2016,2017,2018,2019,2020,Percent Change (1970-2020),Absolute Change (1970-2020),Percent Change (2019-2020),Absolute Change (2019-2020)
0,Alabama,102.646851,98.461114,104.932504,109.563135,108.777543,107.779346,108.089155,111.683852,106.629516,...,119.200889,113.983783,108.594378,112.355471,106.250752,98.308585,-0.042264,-4.338266,-0.074749,-7.942167
1,Alaska,11.348910,12.636423,13.420588,12.490564,12.779110,14.524477,15.969357,17.950301,19.482875,...,35.027804,33.405611,33.729130,34.514589,34.259440,35.967527,2.169250,24.618617,0.049857,1.708086
2,Arizona,24.906189,26.998731,30.179241,34.448720,36.737072,38.221132,43.777153,50.500706,49.284884,...,94.978784,90.855626,90.476141,94.095509,92.537025,80.136708,2.217542,55.230519,-0.134004,-12.400318
3,Arkansas,36.178889,35.091287,37.189109,40.829866,39.112512,36.365524,38.857529,41.649053,42.418586,...,59.053365,62.126250,64.176257,70.786146,65.072202,54.742746,0.513113,18.563857,-0.158738,-10.329456
4,California,294.366786,305.827812,312.715431,329.277423,304.462225,311.476523,326.893420,354.474826,345.235196,...,351.408516,353.356132,356.516548,358.594718,358.164373,303.406714,0.030710,9.039928,-0.152884,-54.757659


In [27]:
df_co2_per_capita.head()

,State,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2015,2016,2017,2018,2019,2020,Percent Change (1970-2020),Absolute Change (1970-2020),Percent Change (2019-2020),Absolute Change (2019-2020)
0,Alabama,29.744089,28.155881,29.650326,30.604228,29.999322,29.295827,28.939533,29.545993,27.826074,...,24.552191,23.419721,22.262070,22.967185,21.648482,19.563897,-0.342259,-10.180192,-0.096292,-2.084585
1,Alaska,37.331940,39.988680,41.421568,37.735844,37.475395,38.628928,39.823833,44.541691,48.105863,...,47.463149,44.960445,45.518394,46.831192,46.674987,49.135965,0.316191,11.804026,0.052726,2.460979
2,Arizona,13.898543,14.239837,15.029503,16.218795,16.525898,16.726973,18.660338,20.825033,19.596375,...,13.900012,13.082164,12.837137,13.134493,12.690212,11.164211,-0.196735,-2.734332,-0.120250,-1.526001
3,Arkansas,18.726133,17.794770,18.419569,19.829950,18.616141,16.835891,17.906695,18.854257,18.911541,...,19.816565,20.764121,21.363601,23.501377,21.539954,18.174882,-0.029437,-0.551250,-0.156225,-3.365072
4,California,14.713190,15.031348,15.191422,15.778304,14.379060,14.461720,14.902144,15.858752,15.118024,...,9.032709,9.025930,9.062905,9.092850,9.081707,7.681183,-0.477939,-7.032007,-0.154214,-1.400525


In [28]:
# clean data: CO2 emissions
# drop unnecessary columns, null values, and irrelevant years
co2ColumnsToDrop = [
    'Percent Change (1970-2020)', 'Absolute Change (1970-2020)',
    'Percent Change (2019-2020)', 'Absolute Change (2019-2020)']

co2ColumnsToDrop.extend([col for col in df_co2.columns
                         if type(col) == int
                         and (col < YEAR_START or col > YEAR_END)])

df_co2.drop(co2ColumnsToDrop, axis = 1, inplace = True)
df_co2.dropna(inplace = True)
df_co2_per_capita.drop(co2ColumnsToDrop, axis = 1, inplace = True)
df_co2_per_capita.dropna(inplace = True)

In [29]:
df_co2.head()

,State,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,Alabama,125.295396,123.222498,130.996584,137.220804,134.133290,133.524213,135.835298,142.298577,133.321950,...,143.532081,145.768722,147.287643,139.359998,119.799631,132.488047,129.546189,122.631371,120.463436,122.632268
1,Alaska,35.692887,35.546771,40.038119,41.115701,41.226671,42.241038,42.863389,43.579298,42.552855,...,47.490384,45.422103,43.653346,39.079026,37.286707,37.137284,37.085337,36.155281,34.049190,33.919405
2,Arizona,69.071835,71.791268,66.738602,68.580736,71.730126,76.724966,80.623874,86.640039,88.861709,...,97.299157,100.503816,102.353396,102.612853,93.865336,99.486477,97.661372,95.453669,99.308288,97.293281
3,Arkansas,50.434861,54.303380,57.665773,60.193352,59.150996,60.529906,62.721860,63.340678,62.497584,...,60.236705,62.097951,63.377572,64.170832,61.543766,66.087436,67.484337,66.255250,68.512147,68.913761
4,California,342.287923,359.306524,348.747120,349.802927,352.665600,362.665776,366.095498,382.283109,385.555775,...,389.574526,397.784683,402.552358,383.998202,370.354625,356.588090,342.651288,348.745644,349.706025,345.376602


In [30]:
df_co2_per_capita.head()

,State,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,Alabama,29.733127,28.925469,30.485591,31.683400,30.708171,30.311967,30.662595,31.962843,29.839290,...,31.407458,31.490327,31.518862,29.537939,25.178569,27.682417,26.988789,25.458038,24.930347,25.316323
1,Alaska,59.587457,58.949869,66.288277,67.513467,67.253950,68.130706,68.581422,69.393787,67.118068,...,71.199975,67.292004,64.196097,56.883589,53.342928,52.013003,51.364733,49.460029,46.137113,46.023616
2,Arizona,16.991841,16.911959,15.058349,14.951109,15.142522,15.712670,16.047746,16.787452,16.852211,...,16.663668,16.670064,16.594260,16.339626,14.798256,15.527778,15.087498,14.559742,14.967338,14.450213
3,Arkansas,20.535367,21.773609,22.747839,23.403325,22.741636,23.050231,23.650777,23.643404,23.216042,...,21.660088,22.004944,22.245550,22.320289,21.243965,22.617192,22.946051,22.436590,23.145996,23.211102
4,California,10.944458,11.412353,11.002528,10.924855,10.855926,10.993870,10.928550,11.247591,11.182336,...,10.873466,11.043133,11.104893,10.490608,10.020146,9.554879,9.104349,9.190819,9.141685,8.950595


In [31]:
# save to database: CO2 emissions
df_co2.to_sql("co2_emissions", con, if_exists = "replace", index = False)
df_co2_per_capita.to_sql("co2_emissions_per_capita", con, if_exists = "replace", index = False)

52

In [32]:
# verify database population
query_co2_emissions_head = """
SELECT *
FROM co2_emissions
LIMIT 6;
"""

df_co2_emissions_head = pd.read_sql_query(query_co2_emissions_head, con)
df_co2_emissions_head

,State,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,Alabama,125.295396,123.222498,130.996584,137.220804,134.133290,133.524213,135.835298,142.298577,133.321950,...,143.532081,145.768722,147.287643,139.359998,119.799631,132.488047,129.546189,122.631371,120.463436,122.632268
1,Alaska,35.692887,35.546771,40.038119,41.115701,41.226671,42.241038,42.863389,43.579298,42.552855,...,47.490384,45.422103,43.653346,39.079026,37.286707,37.137284,37.085337,36.155281,34.049190,33.919405
2,Arizona,69.071835,71.791268,66.738602,68.580736,71.730126,76.724966,80.623874,86.640039,88.861709,...,97.299157,100.503816,102.353396,102.612853,93.865336,99.486477,97.661372,95.453669,99.308288,97.293281
3,Arkansas,50.434861,54.303380,57.665773,60.193352,59.150996,60.529906,62.721860,63.340678,62.497584,...,60.236705,62.097951,63.377572,64.170832,61.543766,66.087436,67.484337,66.255250,68.512147,68.913761
4,California,342.287923,359.306524,348.747120,349.802927,352.665600,362.665776,366.095498,382.283109,385.555775,...,389.574526,397.784683,402.552358,383.998202,370.354625,356.588090,342.651288,348.745644,349.706025,345.376602
5,Colorado,72.494168,73.008631,73.059672,76.011772,76.150940,78.417977,80.458152,85.516380,93.301223,...,95.751760,96.625992,99.331120,97.387166,93.326193,95.887281,92.505559,91.206979,92.137212,92.772031


# Visualizations

## Nation-Wide Visualization

In [33]:
query_over_time = """
SELECT fuel_type,
  AVG(`1993`) AS "1993",
  AVG(`1994`) AS "1994",
  AVG(`1995`) AS "1995",
  AVG(`1996`) AS "1996",
  AVG(`1997`) AS "1997",
  AVG(`1998`) AS "1998",
  AVG(`1999`) AS "1999",
  AVG(`2000`) AS "2000",
  AVG(`2001`) AS "2001",
  AVG(`2002`) AS "2002",
  AVG(`2003`) AS "2003",
  AVG(`2004`) AS "2004",
  AVG(`2005`) AS "2005",
  AVG(`2006`) AS "2006",
  AVG(`2007`) AS "2007",
  AVG(`2008`) AS "2008",
  AVG(`2009`) AS "2009",
  AVG(`2010`) AS "2010",
  AVG(`2011`) AS "2011",
  AVG(`2012`) AS "2012",
  AVG(`2013`) AS "2013",
  AVG(`2014`) AS "2014"
FROM {table}
GROUP BY {column}
HAVING {column} = "{filter}";
"""

# Nationwide avg. % vehicles powered by clean fuel in each year
df_clean_fuel_over_time = pd.read_sql_query(query_over_time.format(table = "vehicles", column = "fuel_type", filter = "Clean Fuel"), con)
df_clean_fuel_over_time

,fuel_type,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,Clean Fuel,0.041111,0.041111,0.041111,0.041111,0.041111,0.041111,0.041111,0.365078,0.614039,...,4.009627,4.882078,7.257882,4.716922,4.706216,8.292961,3.537882,4.596627,5.014451,4.630824


In [34]:
# Nationwide avg. % vehicles powered by gas in each year
df_gas_powered_over_time = pd.read_sql_query(query_over_time.format(table = "vehicles", column = "fuel_type", filter = "Gas Powered"), con)
df_gas_powered_over_time

,fuel_type,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,Gas Powered,98.665706,98.665706,98.665706,98.665706,98.665706,98.665706,98.665706,98.648843,97.625588,...,95.115922,94.26898,91.889137,94.406196,94.275412,90.692137,95.735314,94.613824,94.197216,94.445118


In [35]:
query_gasoline_over_time = """
SELECT *
FROM gasoline
WHERE State = "US";
"""

# Nationwide avg. gasoline price each year
df_gasoline_over_time = pd.read_sql_query(query_gasoline_over_time, con)
df_gasoline_over_time

,State,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,US,8.89,8.99,9.24,9.86,9.83,8.47,9.32,11.91,11.36,...,17.98,20.4,22.34,26.09,18.97,22.64,28.59,29.42,28.61,27.49


In [36]:
query_emissions_over_time = """
SELECT *
FROM co2_emissions_per_capita
WHERE State = "Average all states";
"""

# Nationwide total CO2 emissions per capita in each year
df_emissions_over_time = pd.read_sql_query(query_emissions_over_time, con)
df_emissions_over_time

,State,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,Average all states,19.886405,19.917408,19.923221,20.42729,20.452208,20.346701,20.341495,20.785291,20.196397,...,20.266309,19.793594,19.905915,19.084457,17.588661,18.055417,17.490495,16.660718,16.961662,17.014198


In [37]:
# Compile all averages into single dataframe
# No scaling is applied here, as the ranges are comparable for 3/4 of the graphs (all but "Avg. % Vehicles Powered by Gas").
# This is also the reason no unit label is applied to the y-axis in the graph generated in the following kernel. Please see the legend for the units for each graph.
df_time = pd.DataFrame()
df_time = pd.concat([df_clean_fuel_over_time, df_gas_powered_over_time, df_gasoline_over_time, df_emissions_over_time], ignore_index = True)
df_time.drop(["fuel_type", "State"], axis = 1, inplace = True)
df_time["Metric"] = ["Avg. % Vehicles Powered by Clean Fuel", "Avg. % Vehicles Powered by Gas", "Avg. $ Per Gasoline Barrel", "Metric Tons of CO2 Emissions Per Capita"]

df_time

,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,Metric
0,0.041111,0.041111,0.041111,0.041111,0.041111,0.041111,0.041111,0.365078,0.614039,0.732373,...,4.882078,7.257882,4.716922,4.706216,8.292961,3.537882,4.596627,5.014451,4.630824,Avg. % Vehicles Powered by Clean Fuel
1,98.665706,98.665706,98.665706,98.665706,98.665706,98.665706,98.665706,98.648843,97.625588,96.460608,...,94.268980,91.889137,94.406196,94.275412,90.692137,95.735314,94.613824,94.197216,94.445118,Avg. % Vehicles Powered by Gas
2,8.890000,8.990000,9.240000,9.860000,9.830000,8.470000,9.320000,11.910000,11.360000,10.710000,...,20.400000,22.340000,26.090000,18.970000,22.640000,28.590000,29.420000,28.610000,27.490000,Avg. $ Per Gasoline Barrel
3,19.886405,19.917408,19.923221,20.427290,20.452208,20.346701,20.341495,20.785291,20.196397,20.143114,...,19.793594,19.905915,19.084457,17.588661,18.055417,17.490495,16.660718,16.961662,17.014198,Metric Tons of CO2 Emissions Per Capita


In [38]:
# Generate graph of nationwide averages

fig = go.Figure()

for row in df_time.itertuples(index = False):
  trace = go.Scatter(x = df_time.drop("Metric", axis = 1).columns, y = row, name = row.Metric, mode = 'lines')

  # Do not show this trace by default, as trends are much more apprarent while it is toggled off (due to difference in ranges)
  # Can be enabled by clicking the trace in the legened: single-click to include, double-click to isolate
  if (row.Metric == "Avg. % Vehicles Powered by Gas"):
    trace.visible = "legendonly"

  fig.add_trace(trace)

fig.update_layout(title = "Nationwide Averages (1993 - 2014)",
                  xaxis_title = 'Year',
                  width = 1200)

fig.show()

## State-Wise Visualizations

In [39]:
# The code block performs the following operations:
# - Creates a list of available states by extracting the unique values from the 'state_abbr' column of the 'df_vehicles' DataFrame.
# - Extracts the years from the 'df_vehicles' DataFrame by selecting the column names starting from index 2 and converts them to a list.
# - Defines a function named 'generate_plot' that takes a 'state_selection' parameter representing the selected states.
# - Within the 'generate_plot' function:
    # - Filters the 'df_vehicles' DataFrame based on the selected states.
    # - Creates an empty figure using the Plotly library.
    # - Iterates over the unique fuel types in the filtered data.
    # - For each fuel type, iterates over the unique states in the filtered data.
    # - Filters the data for the current fuel type and state.
    # - Extracts the sales data for the state and converts it to a list.
    # - Adds a line trace to the figure for the current state and fuel type using the extracted sales data.
    # - Updates the layout of the figure, setting the title, x-axis title, and y-axis title.
    # Displays the plot using the 'show' method of the figure object.
# - Calls the 'generate_plot' function with the list of available states to generate the initial plot with all states selected.

# These operations result in an interactive visualization showing the vehicle inventory sales by state and fuel type over the available years.
# The plot consists of multiple line traces, where each trace represents a specific state and fuel type combination.
# The plot is displayed using the Plotly library.

# Create a list of available states
available_states = df_vehicles['state_abbr'].unique()

# Extract the years from the df_gas dataframe
years = df_vehicles.columns[2:].tolist()

# Create a function to generate the interactive visualization
def generate_plot(state_selection):
    # Filter the data based on the selected states
    filtered_data = df_vehicles[df_vehicles['state_abbr'].isin(state_selection)]

    # Create an empty figure
    fig = go.Figure()

    # Iterate over the fuel types
    for fuel_type in filtered_data['fuel_type'].unique():
        # Filter the data for the current fuel type
        fuel_data = filtered_data[filtered_data['fuel_type'] == fuel_type]

        # Iterate over the states
        for state in fuel_data['state_abbr'].unique():
            # Filter the data for the current state
            state_data = fuel_data[fuel_data['state_abbr'] == state]

            # Extract the sales data for the state
            sales_data = state_data.iloc[:, 2:].values.tolist()[0]

            # Add a line trace for the current state and fuel type
            fig.add_trace(go.Scatter(
                x=years,
                y=sales_data,
                mode='lines',
                name=f"{state} - {fuel_type}"
            ))

    # Set the layout of the chart
    fig.update_layout(
        title='Vehicle Inventory Sales by State and Fuel Type',
        xaxis_title='Year',
        yaxis_title='Sales (%)'
    )

    # Show the plot
    fig.show()

# Generate the initial plot with all states selected
generate_plot(available_states)

In [40]:
# The code block performs the following operations:
# - Creates a list of available states and fuel types by extracting the unique values from the 'state_abbr' and 'fuel_type' columns of the 'df_vehicles' DataFrame.
# - Creates a widget for state selection using the SelectMultiple class from the widgets module, allowing the user to select multiple states.
# - Defines a function named 'on_change' to handle the value change event of the state selection widget.
# - Registers the 'on_change' function as the event handler for the state selection widget.
# - Creates a widget for fuel type selection using the SelectMultiple class, with the initial value set to all available fuel types.
# - Defines a function named 'generate_plot' that takes 'state_selection' and 'fuel_type_selection' as parameters.
# - Within the 'generate_plot' function:
    # - Filters the 'df_vehicles' DataFrame based on the selected states and fuel types.
    # - Creates a subplot for each selected state using the 'make_subplots' function from the Plotly library.
    # - Iterates over the selected states and fuel types to generate line traces for each combination.
    # - Adds a line trace to the subplot for the current state and fuel type, with appropriate settings.
    # - Updates the layout of the figure, including the title, x-axis title, y-axis title, and height.
    # - Displays the plot using the 'show' method of the figure object.
# - Generates the initial plot with the selected initial states and all available fuel types.
# - Displays the state and fuel type selection widgets to allow the user to interact with the plot.

# These operations create an interactive visualization where the user can select multiple states and fuel types to display the vehicle inventory sales over time.
# The plot updates dynamically based on the user's selections.
# The state selection and fuel type selection widgets enable interactive filtering of the data and provide a responsive user interface.

# Create a list of available states and fuel types
available_states = df_vehicles['state_abbr'].unique()
available_fuel_types = df_vehicles['fuel_type'].unique()

# Create checkboxes for state selection
state_selection_widget = widgets.SelectMultiple(
    options=available_states,
    description='Select States:'
)

# Create a function to handle widget value change event
def on_change(change):
    if change['name'] == 'value':
        state_selection = state_selection_widget.value
        fuel_type_selection = fuel_type_selection_widget.value
        generate_plot(state_selection, fuel_type_selection)

# Register the on_change function as the event handler for state selection
state_selection_widget.observe(on_change)

# Create checkboxes for fuel type selection
fuel_type_selection_widget = widgets.SelectMultiple(
    options=available_fuel_types,
    value=tuple(available_fuel_types),
    description='Select Fuel Types:'
)

# Create a function to generate the interactive visualization
def generate_plot(state_selection, fuel_type_selection):
    # Filter the data based on the selected states and fuel types
    filtered_data = df_vehicles[
        (df_vehicles['state_abbr'].isin(state_selection)) &
        (df_vehicles['fuel_type'].isin(fuel_type_selection))
    ]

    # Create a subplot for each state
    fig = make_subplots(rows=len(state_selection), cols=1, shared_xaxes=True, subplot_titles=state_selection)

    # Iterate over the selected states
    for i, state in enumerate(state_selection):
        # Filter the data for the current state
        state_data = filtered_data[filtered_data['state_abbr'] == state]

        # Iterate over the selected fuel types
        for fuel_type in fuel_type_selection:
            # Filter the data for the current fuel type
            fuel_data = state_data[state_data['fuel_type'] == fuel_type]

            # Extract the sales data for the state and fuel type
            sales_data = fuel_data.iloc[:, 2:].values.tolist()[0]

            # Add a line trace for the current state and fuel type
            fig.add_trace(go.Scatter(
                x=years,
                y=sales_data,
                mode='lines',
                name=f"{state} - {fuel_type}",
                legendgroup=f"{state}",
                showlegend=(i == 0)  # Show legend only for the first subplot
            ), row=i+1, col=1)

    # Set the layout of the chart
    fig.update_layout(
        title='Vehicle Inventory Sales by State and Fuel Type',
        xaxis_title='Year',
        yaxis_title='Sales (%)',
        height=600 * len(state_selection)
    )

    # Show the plot
    fig.show()

# Generate the initial plot with all states and fuel types selected
initial_states = ['CA']
generate_plot(initial_states, available_fuel_types)

# Display the state and fuel type selection widgets
display(state_selection_widget, fuel_type_selection_widget)

SelectMultiple(description='Select States:', options=('AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'F…

SelectMultiple(description='Select Fuel Types:', index=(0, 1, 2), options=('Clean Fuel', 'Gas Powered', 'Other…

In [43]:
# The code block performs the following operations:
# - Import necessary libraries: pandas, plotly.express, ipywidgets, and IPython.display.
# - Define a dropdown widget for state selection, using the unique values from the 'state_abbr' column of the 'df_vehicles' DataFrame. The default value is set to 'CA'.
# - Define a dropdown widget for year selection, using the available years from the DataFrame columns. The default value is set to 2014.
# - Define a button widget for generating the chart.
# - Define a function named 'generate_chart' to generate the chart based on the selected state and year.
# - Within the 'generate_chart' function:
#     - Get the selected state and year from the dropdown widgets.
#     - Filter the 'df_vehicles' DataFrame based on the selected state.
#     - Extract the 'fuel_type' and selected year columns from the filtered DataFrame.
#     - Convert the column names to strings.
#     - Create a pie chart using the filtered data, with the values representing the selected year and the names representing the fuel types. The chart's title includes the selected state and year.
#     - Show the chart using the 'show' method of the figure object.
# - Attach the 'generate_chart' function as the click event handler for the generate button.
# - Call the 'generate_chart' function initially with None as the argument to generate the chart with the default values.
# - Display the state dropdown, year dropdown, and generate button widgets.

# These operations create an interactive chart where the user can select a state and year, and the chart dynamically updates based on the selections.
# The dropdown widgets allow the user to choose the state and year, and the generate button triggers the chart generation.
# The chart is displayed using the Plotly library's 'px.pie' function, showing the vehicle sales by fuel type for the selected state and year.

# Define a dropdown widget for the state selection with default value
states = df_vehicles['state_abbr'].unique()
state_dropdown = widgets.Dropdown(options=states, value='CA', description='State:')

# Define a dropdown widget for the year selection with default value
years = df_vehicles.columns[2:].astype(int)  # Assuming the columns starting from index 2 represent the years
year_dropdown = widgets.Dropdown(options=years, value=2014, description='Year:')

# Define a button widget for triggering the chart generation
generate_button = widgets.Button(description='Generate Chart')

# Define a function to generate the chart based on the selected state and year
def generate_chart(_):
    selected_state = state_dropdown.value
    selected_year = year_dropdown.value

    # Filter dataframe
    filtered_df = df_vehicles[(df_vehicles['state_abbr'] == selected_state)]
    fuel_type_counts = filtered_df[['fuel_type', selected_year]]

    # Convert column names to strings
    fuel_type_counts.columns = fuel_type_counts.columns.astype(str)

    # Create pie chart
    fig = px.pie(fuel_type_counts, values=fuel_type_counts[str(selected_year)], names=fuel_type_counts['fuel_type'], title=f'Vehicle Sales by Fuel Type for the State of {selected_state} in {selected_year}')
    fig.show()

generate_button.on_click(generate_chart)

# Call the generate_chart function initially with default values
generate_chart(None)

# Display the widgets
display(state_dropdown, year_dropdown, generate_button)

Dropdown(description='State:', index=4, options=('AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', '…

Dropdown(description='Year:', index=21, options=(1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2…

Button(description='Generate Chart', style=ButtonStyle())

## Geospatial Visualizations

In [44]:
# Load US States Geometry
us_states = gpd.read_file('https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json')

# Add state abbreviations to the DataFrame
state_abbr = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}
us_states['abbreviation'] = us_states['name'].map(state_abbr)

# Assume df_gas is your DataFrame containing the gasoline prices

# Melt the DataFrame to long format
df_gas_long = df_gas.melt(id_vars='State', var_name='Year', value_name='Price')

# Ensure 'Year' column is of type int for future merges or comparisons
df_gas_long['Year'] = df_gas_long['Year'].astype(int)

# Filter the data for the year 1993
df_gas_1993 = df_gas_long[df_gas_long['Year'] == 1993]

# Now merge the dataframes
us_states = us_states.merge(df_gas_1993, how="left", left_on="abbreviation", right_on="State")

# Define the choropleth color scale using the 'Price' column
color_scale = px.colors.sequential.Plasma

# Create choropleth trace
choropleth = go.Choropleth(
    geojson=us_states.geometry.__geo_interface__,
    locations=us_states.index,
    z=us_states['Price'],
    hovertext=us_states.apply(lambda row: f"{row['abbreviation']}: ${row['Price']}", axis=1),
    colorscale=color_scale,
    colorbar=dict(
        title='Gas Prices',
        len=0.6,
        yanchor='middle',
        y=0.5,
        tickformat=".2f"
    )
)

# Create scatter geo for state center points to display state abbreviations
us_states['center'] = us_states['geometry'].centroid
us_states_points = us_states.copy()
us_states_points.set_geometry("center", inplace = True)

# Compute text positions
text_positions = us_states_points.geometry.apply(lambda geom: geom.representative_point().coords[:])
text_positions = [coords[0] for coords in text_positions]

us_states_points['center_lon'] = [coords[0] for coords in text_positions]
us_states_points['center_lat'] = [coords[1] for coords in text_positions]


scatter_geo = go.Scattergeo(
    lat=us_states_points['center_lat'],
    lon=us_states_points['center_lon'],
    text=us_states_points.apply(lambda row: f"{row['abbreviation']}", axis=1),  # Display both state abbreviation and gas price
    hoverinfo="text",
    mode='text',
    textfont=dict(
        size=10,
        color="black"
    )
)

fig = go.Figure(data=[choropleth, scatter_geo])

fig.update_geos(showcountries=False, showcoastlines=True, showland=True, fitbounds="locations")

# Increase the size of the plot
fig.update_layout(width=1100, height=900)

# Set title with bold font
fig.update_layout(
    title={
        'text': "<b>Gasoline Price (Per Barrel) - 1993</b>",
        'font': {'size': 20}
    }
)

fig.show()


<ipython-input-44-212b83590ead>:52: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [45]:
# Load US States Geometry
us_states = gpd.read_file('https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json')

# Add state abbreviations to the DataFrame
state_abbr = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}
us_states['abbreviation'] = us_states['name'].map(state_abbr)

# Assume df_gas is your DataFrame containing the gasoline prices

# Melt the DataFrame to long format
df_gas_long = df_gas.melt(id_vars='State', var_name='Year', value_name='Price')

# Ensure 'Year' column is of type int for future merges or comparisons
df_gas_long['Year'] = df_gas_long['Year'].astype(int)

# Filter the data for the year 2014
df_gas_2014 = df_gas_long[df_gas_long['Year'] == 2014]

# Now merge the dataframes
us_states = us_states.merge(df_gas_2014, how="left", left_on="abbreviation", right_on="State")

# Define the choropleth color scale using the 'Price' column
color_scale = px.colors.sequential.Plasma

# Create choropleth trace
choropleth = go.Choropleth(
    geojson=us_states.geometry.__geo_interface__,
    locations=us_states.index,
    z=us_states['Price'],
    hovertext=us_states.apply(lambda row: f"{row['abbreviation']}: ${row['Price']}", axis=1),
    colorscale=color_scale,
    colorbar=dict(
        title='Gas Prices',
        len=0.6,
        yanchor='middle',
        y=0.5,
        tickformat=".2f"
    )
)

# Create scatter geo for state center points to display state abbreviations
us_states['center'] = us_states['geometry'].centroid
us_states_points = us_states.copy()
us_states_points.set_geometry("center", inplace = True)

# Compute text positions
text_positions = us_states_points.geometry.apply(lambda geom: geom.representative_point().coords[:])
text_positions = [coords[0] for coords in text_positions]

us_states_points['center_lon'] = [coords[0] for coords in text_positions]
us_states_points['center_lat'] = [coords[1] for coords in text_positions]


scatter_geo = go.Scattergeo(
    lat=us_states_points['center_lat'],
    lon=us_states_points['center_lon'],
    text=us_states_points.apply(lambda row: f"{row['abbreviation']}", axis=1),  # Display both state abbreviation and gas price
    hoverinfo="text",
    mode='text',
    textfont=dict(
        size=10,
        color="black"
    )
)

fig = go.Figure(data=[choropleth, scatter_geo])

fig.update_geos(showcountries=False, showcoastlines=True, showland=True, fitbounds="locations")

# Increase the size of the plot
fig.update_layout(width=1100, height=900)

# Set title with bold font
fig.update_layout(
    title={
        'text': "<b>Gasoline Price (Per Barrel) - 2014</b>",
        'font': {'size': 20}
    }
)

fig.show()


<ipython-input-45-bb72b4d1a1eb>:52: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [46]:
# Load US States Geometry
us_states = gpd.read_file('https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json')

# Add state abbreviations to the DataFrame
state_abbr = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}
us_states['abbreviation'] = us_states['name'].map(state_abbr)

# Assume df_gas is your DataFrame containing the gasoline prices

# Melt the DataFrame to long format
df_gas_long = df_gas.melt(id_vars='State', var_name='Year', value_name='Price')

# Ensure 'Year' column is of type int for future merges or comparisons
df_gas_long['Year'] = df_gas_long['Year'].astype(int)

# Calculate the average fuel price for all years
df_gas_avg = df_gas_long.groupby('State')['Price'].mean().reset_index()

# Now merge the dataframes
us_states = us_states.merge(df_gas_avg, how="left", left_on="abbreviation", right_on="State")

# Define the choropleth color scale using the 'Price' column
color_scale = px.colors.sequential.Plasma

# Create choropleth trace
choropleth = go.Choropleth(
    geojson=us_states.geometry.__geo_interface__,
    locations=us_states.index,
    z=us_states['Price'],
    hovertext=us_states.apply(lambda row: f"{row['abbreviation']}: ${row['Price']}", axis=1),
    colorscale=color_scale,
    colorbar=dict(
        title='Gas Prices',
        len=0.6,
        yanchor='middle',
        y=0.5,
        tickformat=".2f"
    )
)

# Create scatter geo for state center points to display state abbreviations
us_states['center'] = us_states['geometry'].centroid
us_states_points = us_states.copy()
us_states_points.set_geometry("center", inplace=True)

# Compute text positions
text_positions = us_states_points.geometry.apply(lambda geom: geom.representative_point().coords[:])
text_positions = [coords[0] for coords in text_positions]

us_states_points['center_lon'] = [coords[0] for coords in text_positions]
us_states_points['center_lat'] = [coords[1] for coords in text_positions]

scatter_geo = go.Scattergeo(
    lat=us_states_points['center_lat'],
    lon=us_states_points['center_lon'],
    text=us_states_points.apply(lambda row: f"{row['abbreviation']}", axis=1),  # Display state abbreviation
    hoverinfo="text",
    mode='text',
    textfont=dict(
        size=10,
        color="black"
    )
)

fig = go.Figure(data=[choropleth, scatter_geo])

fig.update_geos(showcountries=False, showcoastlines=True, showland=True, fitbounds="locations")

# Increase the size of the plot
fig.update_layout(width=1100, height=900)

# Set title with bold font
fig.update_layout(
    title={
        'text': "<b>Average Fuel Prices (1993-2014)</b>",
        'font': {'size': 20}
    }
)

fig.show()


<ipython-input-46-806c8f4684f5>:52: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [50]:
# Load US States Geometry
us_states = gpd.read_file('https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json')

# Add state abbreviations to the DataFrame
state_abbr = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}
us_states['abbreviation'] = us_states['name'].map(state_abbr)

# Assume df_gas is your DataFrame containing the gasoline prices

# Melt the DataFrame to long format
df_gas_long = df_gas.melt(id_vars='State', var_name='Year', value_name='Price')

# Ensure 'Year' column is of type int for future merges or comparisons
df_gas_long['Year'] = df_gas_long['Year'].astype(int)

# Create the list of years for the slider
years = df_gas_long['Year'].unique()

# Initialize the figure and set the layout
fig = go.Figure()
fig.update_geos(showcountries=False, showcoastlines=True, showland=True, fitbounds="locations")
fig.update_layout(width=1100, height=900)

# Add choropleth and scattergeo for each year
for year in years:
    # Filter the data for the current year
    df_gas_year = df_gas_long[df_gas_long['Year'] == year]
    us_states_year = us_states.merge(df_gas_year, how="left", left_on="abbreviation", right_on="State")

    # Create choropleth trace for the current year
    choropleth = go.Choropleth(
        geojson=us_states_year.geometry.__geo_interface__,
        locations=us_states_year.index,
        z=us_states_year['Price'],
        hovertext=us_states_year.apply(lambda row: f"{row['abbreviation']}: ${row['Price']}", axis=1),
        colorscale=px.colors.sequential.Plasma,
        colorbar=dict(
            title='Gas Prices',
            len=0.6,
            yanchor='middle',
            y=0.35,  # Adjust the y position of the color bar
            tickformat=".2f"
        ),
        showscale=True  # Set showscale to True to display the color bar
    )

    # Create scattergeo trace for state center points
    us_states_year['center'] = us_states_year['geometry'].centroid
    us_states_points = us_states_year.copy()
    us_states_points.set_geometry("center", inplace=True)

    text_positions = us_states_points.geometry.apply(lambda geom: geom.representative_point().coords[:])
    text_positions = [coords[0] for coords in text_positions]

    scatter_geo = go.Scattergeo(
        lat=[coords[1] for coords in text_positions],
        lon=[coords[0] for coords in text_positions],
        text=us_states_points['abbreviation'],
        hoverinfo="text",
        mode='text',
        textfont=dict(
            size=10,
            color="black"
        ),
        showlegend=False
    )

    # Add choropleth and scattergeo traces to the figure
    fig.add_trace(choropleth)
    fig.add_trace(scatter_geo)

# Configure the slider
steps = []
for i, year in enumerate(years):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)}, {"title": f"<b>Gasoline Price (Per Barrel) - {year}</b>"}],
        label=str(year)
    )
    step["args"][0]["visible"][i * 2] = True  # Show choropleth trace
    step["args"][0]["visible"][i * 2 + 1] = True  # Show scattergeo trace
    steps.append(step)

sliders = [
    dict(
        active=0,
        currentvalue={"prefix": "Year: "},
        pad={"t": 50},
        steps=steps,
        ticklen=10  # Set the length of the ticks
    )
]

fig.update_layout(sliders=sliders)


# Function to handle key press event
def on_key(event):
    if event.key == 'right':
        slider.value = (slider.value + 1) % len(steps)
    elif event.key == 'left':
        slider.value = (slider.value - 1) % len(steps)


# Create the slider using ipywidgets
slider = widgets.IntSlider(min=0, max=len(steps) - 1, value=0, continuous_update=False)
slider.observe(lambda change: fig.update_traces(visible=[False] * len(fig.data)), 'value')

# Display the figure and slider
display(fig)
display(slider)

# Attach the key press event handler
plt.gcf().canvas.mpl_connect('key_press_event', on_key)

# Save animation frames as PNG images
frame_dir = "animation_frames"
os.makedirs(frame_dir, exist_ok=True)
for i, frame in enumerate(fig.frames):
    path = os.path.join(frame_dir, f"frame_{i}.png")
    pio.write_image(frame.data, path)

# Create the animated GIF using external tools or libraries

# For example, using ImageMagick:
# magick convert -delay 100 -loop 0 animation_frames/*.png animation.gif

# Or using imageio:
frames_path = [os.path.join(frame_dir, f"frame_{i}.png") for i in range(len(fig.frames))]
images = [imageio.imread(path) for path in frames_path]
imageio.mimsave("animation.gif", images, duration=0.5)

# Remove the animation frames directory
# for path in frames_path:
#     os.remove(path)
# os.rmdir(frame_dir)


Output hidden; open in https://colab.research.google.com to view.

# Teardown

In [ ]:
# close database connection
con.close()